In [ ]:
from huggingface_hub import login
login()

In [ ]:
!git clone https://github.com/huggingface/parler-tts.git
%cd parler-tts
!pip install --quiet -e .[train]

In [ ]:
!pip install --upgrade protobuf wandb==0.16.6

In [ ]:
# Import the library
import wandb

# Set the W&B API Key
import os
os.environ['WANDB_API_KEY'] = '7abff37dc0360f7fcdbcf490b6327bf94bed0880'

# Login to W&B
wandb.login()

In [ ]:
import subprocess
# Define the command you want to run
command = [
    "accelerate", "launch", "./training/run_parler_tts_training.py",
    "--model_name_or_path", "procit001/parler-tts-mini_fbdata_v1.2",
    "--feature_extractor_name", "parler-tts/dac_44khZ_8kbps",
    "--description_tokenizer_name", "procit001/parler-tts-mini_fbdata_v1.2",
    "--prompt_tokenizer_name", "procit001/parler-tts-mini_fbdata_v1.2",
    "--report_to", "wandb",
    "--overwrite_output_dir", "true",
    "--train_dataset_name", "Procit004/Modified_audio_data_API",
    "--train_metadata_dataset_name", "Procit004/OwnData_english_model_v2",
    "--train_dataset_config_name", "default",
    "--train_split_name", "train",
    "--eval_dataset_name", "Procit004/Modified_audio_data_API",
    "--eval_metadata_dataset_name", "Procit004/OwnData_english_model_v2",
    "--eval_dataset_config_name", "default",
    "--eval_split_name", "train",
    "--max_eval_samples", "8",
    "--per_device_eval_batch_size", "8",
    "--target_audio_column_name", "audio",
    "--description_column_name", "text_description",
    "--prompt_column_name", "text",
    "--max_duration_in_seconds", "20",
    "--min_duration_in_seconds", "2.0",
    "--max_text_length", "400",
    "--preprocessing_num_workers", "2",
    "--do_train", "true",
    "--num_train_epochs", "40",
    "--gradient_accumulation_steps", "18",
    "--gradient_checkpointing", "true",
    "--per_device_train_batch_size", "2",
    "--learning_rate", "0.00008",
    "--adam_beta1", "0.9",
    "--adam_beta2", "0.99",
    "--weight_decay", "0.01",
    "--lr_scheduler_type", "constant_with_warmup",
    "--warmup_steps", "50",
    "--logging_steps", "2",
    "--freeze_text_encoder", "true",
    "--audio_encoder_per_device_batch_size", "4",
    "--dtype", "float16",
    "--seed", "456",
    "--output_dir", "./output_dir_training/",
    "--temporary_save_to_disk", "./audio_code_tmp/",
    "--save_to_disk", "./tmp_dataset_audio/",
    "--dataloader_num_workers", "2",
    "--do_eval",
    "--predict_with_generate",
    "--include_inputs_for_metrics",
    "--group_by_length", "true"
]

# Run the command
process = subprocess.Popen(command, stdin=subprocess.PIPE, text=True)
process.communicate(input="3\n")  # Automatically choose option 3

# Wait for the process to complete
process.wait()

In [ ]:
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("device",device)
model = ParlerTTSForConditionalGeneration.from_pretrained("output_dir_training/", torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained("output_dir_training/")

In [ ]:
prompt = "My name is Rikesh Shah. I live in Kathmandu"
description = "The speech sample is very clear and has a very fast pace, but it is very confined in its sound"

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()

In [ ]:
from IPython.display import Audio
Audio(audio_arr, rate=model.config.sampling_rate)

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.push_to_hub("parler-tts-OwnVoice_v2")
tokenizer.push_to_hub("parler-tts-OwnVoice_v2")